In [ ]:
import time

# Iniciar el temporizador
start_time = time.time()
# Asumiendo que Environment_name y targets están definidos correctamente
# Asegúrate de que `Environment_name` esté definido
database_path = f'../../sqlite_data/{Environment_name}_graph_paths.db'

# Crear la tabla de caminos
with sqlite3.connect(database_path) as connection:
    create_paths_table(connection)

    # Recolectar los caminos y sus costos
    paths_to_insert = []
    for target in targets:
        for path in nx.all_simple_paths(G, source=sources[0], target=target):
            path_cost = sum(G[u][v]["cost"] for u, v in zip(path, path[1:]))
            paths_to_insert.append((sources[0], target, path_cost, path))

    # Insertar los caminos por lotes para mejorar el rendimiento
    with connection:
        connection.executemany(
            "INSERT OR REPLACE INTO paths (source, target, cost, path) VALUES (?, ?, ?, ?)",
            paths_to_insert
        )
end_time = time.time()

# Calcular el tiempo transcurrido
elapsed_time = end_time - start_time

# Mostrar el tiempo que ha tardado
print(f"Tiempo de ejecución: {elapsed_time} segundos")

In [ ]:
shortest_path = nx.dijkstra_path(G, sources[0], target, weight='cost')
shortest_path_cost = nx.dijkstra_path_length(G, sources[0], target, weight='cost')

# Establecer el factor gamma
gamma = 1.5

# Calcular el límite de costo
cost_limit = shortest_path_cost * gamma

# Función para realizar DFS con poda por costo máximo
def dfs_with_pruning(G, source, target, path, cost, cost_limit):
    path.append(source)
    if source == target:
        yield path[:]
    else:
        for neighbor in G.neighbors(source):
            edge_cost = G[source][neighbor].get('cost', 1)
            new_cost = cost + edge_cost
            if new_cost <= cost_limit:
                yield from dfs_with_pruning(G, neighbor, target, path, new_cost, cost_limit)
    path.pop()

# Función para obtener los k caminos más cortos dentro del límite de costo
def k_shortest_paths_within_cost_limit(G, source, target, k, cost_limit):
    paths = []
    for path in dfs_with_pruning(G, source, target, [], 0, cost_limit):
        paths.append(path)
        if len(paths) >= k:
            break
    return paths

# Obtener los k caminos más cortos dentro del límite de costo
k = 3
paths = k_shortest_paths_within_cost_limit(G, sources[0], target, k, cost_limit)

# Mostrar los caminos y sus costos
for path in paths:
    path_cost = sum(G[u][v].get('cost', 1) for u, v in zip(path[:-1], path[1:]))
    print(f"Camino: {path}, Costo: {path_cost}")